In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
import os # batchsize 64
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,max_split_size_mb:128"

In [5]:
import torch
device="cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"



In [7]:
from huggingface_hub import hf_hub_download
model_path = hf_hub_download(repo_id="aliarda/llama-50M-randParams", filename="llama-50M.safetensors", local_dir="/content/")
state_dict = sftorch.load_file(model_path, device=device)

OSError: [Errno 30] Read-only file system: '/content'

In [ ]:
llama_config = LlamaConfig(
    vocab_size=32768,
    emb_dim=256,
    context_length=256
    n_heads=128,
    n_layers=20,
    n_kv_groups=64,
    hidden_dim=2048,
)

llama_model = LlamaForCausalLM(llama_config)
llama_model = llama_model.to(device)
llama_model
#pad eos değistir

In [ ]:
llama_model.load_state_dict(state_dict)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

pad_id = 5
eos_id = 6

DATALOADER


In [ ]:
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, token_ids: list, context_length: int, stride: int):
        super().__init__()

        self.inputs = []
        self.targets = []

        for i in range(0, len(token_ids) - context_length, stride):
            input_chunk = token_ids[i:i + context_length]
            target_chunk = token_ids[i + 1:i + context_length + 1]

            # truncate if the chunk is longer than context_length
            input_chunk = input_chunk[:context_length]
            target_chunk = target_chunk[:context_length]

            # pad the input and target chunks to context_length
            input_chunk += [pad_id] * (context_length - len(input_chunk))
            target_chunk += [pad_id] * (context_length - len(target_chunk))

            # truncate if the chunk is longer than context_length
            input_chunk = input_chunk[:context_length]
            target_chunk = target_chunk[:context_length]

            self.inputs.append(torch.tensor(input_chunk, dtype=torch.long))
            self.targets.append(torch.tensor(target_chunk, dtype=torch.long))

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

In [ ]:
def create_dataloader(token_ids: list, context_len: int, stride: int, batch_size: int, shuffle: bool, device: str = "cpu"):
    dataset = TextDataset(token_ids, context_len, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        generator=torch.Generator(device=device)
    )
    return dataloader

training

In [ ]:
def generate(model, x: torch.Tensor, max_new_tokens: int): # top_k, top_p, temperature
  tokens = x.detach().cpu().numpy().tolist()

  for _ in range(max_new_tokens):
    x = x.unsqueeze(0).to(device)
    out = model.forward(x)
    out = out.squeeze(0)
    probs = torch.softmax(out[-1], dim=-1)
    _, max_index = torch.max(probs, dim=-1)
    tokens.append(max_index.item())
    if max_index == eos_id or len(tokens) > context_len: # <eos> and max context length
      break

    x = torch.tensor(tokens)

  return tokens

In [ ]:
from safetensors.torch import load_file

tensors = load_file("tr_tokenizer_ids.safetensors")#ids from turkish_tokenizer(github alibayram)
token_ids = tensors['a']
print(type(token_ids))  # should be <class 'torch.Tensor'>

<class 'torch.Tensor'>


In [ ]:
train_dataloader = create_dataloader(token_ids.tolist()[:20], 256, 256, 64, False,device)


In [ ]:
import time
import os
import torch
from tqdm import tqdm
from bitsandbytes.optim import AdamW8bit

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = AdamW8bit(llama_model.parameters(), lr=1e-3)

epoch = 2
save_interval = 2 * 60 * 60  # 2 hours in seconds
last_save_time = time.time()

for epoch_idx in range(epoch):
    last_loss = 0
    total_loss = 0
    checkpoint_num = 1  # Counter for intermediate saves in this epoch

    for i, (X, Y) in enumerate(tqdm(train_dataloader)):

        X, Y = X.to(device), Y.to(device)

        pred = llama_model(X)
        loss = loss_fn(pred.flatten(0, 1), Y.flatten())
        total_loss += loss.item()
        last_loss = loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        del pred, loss, X, Y
        torch.cuda.empty_cache()

        # ✅ Check elapsed time
        if time.time() - last_save_time >= save_interval:
            last_save_time = time.time()
            save_path = os.path.join(
                save_dir,
                f"ahmet_tr_tokenizer_50M_llama_model_{epoch_idx+1}_{checkpoint_num}.safetensors"
            )
            torch.save(llama_model.state_dict(), save_path)
            print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Saved intermediate model to {save_path}")
            checkpoint_num += 1  # Increment checkpoint number

    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch_idx + 1} loss: {last_loss.item()} average loss: {average_loss}")

    # Save final model for the epoch as well
    save_path = os.path.join(
        save_dir,
        f"ahmet_tr_tokenizer_50M_llama_model_{epoch_idx+1}_{checkpoint_num}.safetensors"
    )
    torch.save(llama_model.state_dict(), save_path)
    print(f"Epoch completed. Saved model to {save_path}")